In [1]:
from glob import glob

In [8]:
!ls

Model_Evalution.ipynb
Model_Evalution.pdf
adaboost.pkl
classifier_regressor.pkl
deployment_python file.ipynb
dtree_clf.pkl
gnaivebayes.pkl
knn_clf.pkl
logistic_model.pkl
model_evalution_data.csv
pipeline.ipynb
rforest_clf.pkl
svc_model.pkl


In [ ]:
# activities = ["EMI","PROI","ELA"]	

In [23]:
%%writefile prediction.py

import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import pickle

models = {'logistic_model':'logistic_model.pkl', 
#           'classifier_regressor': 'classifier_regressor.pkl', 
          'gnaivebayes': 'gnaivebayes.pkl', 
          'knn_clf': 'knn_clf.pkl',
          'dtree_clf' : 'dtree_clf.pkl', 
           
#           'svc_model': 'svc_model.pkl', 
          'rforest_clf': 'rforest_clf.pkl', 
          'adaboost': 'adaboost.pkl'}


# for key, mod in models.items():
#     with open(mod, 'rb') as f:  # open a model file
#         pickle.load(mod, f) # deserialize the list

pickle_in = open('adaboost.pkl', 'rb') 
adaboost_model = pickle.load(pickle_in)

pickle_in = open('logistic_model.pkl', 'rb') 
LG_model = pickle.load(pickle_in)

pickle_in = open('dtree_clf.pkl', 'rb') 
DT_model = pickle.load(pickle_in)

pickle_in = open('knn_clf.pkl', 'rb') 
KNN_model = pickle.load(pickle_in)

pickle_in = open('svc_model.pkl', 'rb') 
SVC_model = pickle.load(pickle_in)

pickle_in = open('rforest_clf.pkl', 'rb') 
RF_model = pickle.load(pickle_in)

pickle_in = open('gnaivebayes.pkl', 'rb') 
NB_model = pickle.load(pickle_in)




not_default_example = {'DateCreditPulled': 14346,
 'LoanOriginationQuarter': 17,
 'BorrowerAPR': 0.16516,
 'BorrowerRate': 0.158,
 'EstimatedEffectiveYield': 0.1787990044912847,
 'EstimatedReturn': 0.1077559341514328,
 'LoanMonthsSinceOrigination': 78,
 'MonthlyLoanPayment': 5.803416625940581,
 'LP_CustomerPayments': 11396.14,
 'LP_CollectionFees': 0.0,
 'LP_GrossPrincipalLoss': 0.0,
 'LP_NetPrincipalLoss': 0.0,
 'DateCreditPulled_Year': 2007}


default_example = {'DateCreditPulled': 47155,
 'LoanOriginationQuarter': 14,
 'BorrowerAPR': 0.35797,
 'BorrowerRate': 0.3177,
 'EstimatedEffectiveYield': 0.2896,
 'EstimatedReturn': 0.1246,
 'LoanMonthsSinceOrigination': 23,
 'MonthlyLoanPayment': 5.163127456486353,
 'LP_CustomerPayments': 521.13,
 'LP_CollectionFees': 0.0,
 'LP_GrossPrincipalLoss': 3790.25,
 'LP_NetPrincipalLoss': 3790.25,
 'DateCreditPulled_Year': 2012}






def get_prediction(data,model):
    """
    Predict the class of a given data point.
    """
    return model.predict(data)

Overwriting prediction.py


In [25]:
%%writefile app.py


import pickle
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from prediction import get_prediction, models, not_default_example, default_example
from sklearn.preprocessing import RobustScaler



st.set_page_config(page_title='Prosper LoanStatus Prediction App', page_icon="💱",
                               layout="wide", initial_sidebar_state='expanded')


# creating option list for dropdown menu

features = ['ListingNumber', 'ClosedDate', 'BorrowerAPR', 'BorrowerRate','DateCreditPulled', 'LoanMonthsSinceOrigination', 'LoanNumber',
       'LoanOriginationQuarter', 'LP_CustomerPayments','LP_CustomerPrincipalPayments']

st.markdown("<h1 style='color:white ;text-align: center;'>Prosper LoanStatus Prediction App 💸 </h1>", unsafe_allow_html=True)


def prediction(num):
    if num ==0:
        return "Loan is not Defaulted"
    else:
        return "Loan is Defaulted"


def main():
    with st.form('prediction_form'):

        st.header("Predict the input for following features:") 
        st.header("![Alt Text](https://th.bing.com/th/id/OIP.IdEAQWd4unaWF60q6Wni6AAAAA?pid=ImgDet&rs=1)")
        
        activities=['Logistic Regression','Naives Bayes', 'KNN', 'Decision Tree', 'Random Forest', 'Ada Boost']
        model_names = {name:model_name for name, model_name in zip(activities, models.keys())}
        option=st.sidebar.selectbox('Which model would you like to use?' ,activities)
        st.write('You selected:', option)
        st.subheader(option)

        DateCreditPulled = st.number_input('DateCreditPulled:', min_value=0, max_value=57331)
        LoanOriginationQuarter = st.number_input('LoanOriginationQuarter:', min_value=0, max_value=32)
      
        BorrowerAPR = st.number_input('BorrowerAPR:', min_value=0.0, max_value=0.6, step=1.,format="%.2f")
        BorrowerRate = st.number_input ( 'BorrowerRate:', min_value=0.0, max_value=0.5, step=1.,format="%.2f")
        EstimatedEffectiveYield = st.number_input('EstimatedEffectiveYield:', min_value=-0.2, max_value=0.4, step=1.,format="%.2f")
        EstimatedReturn = st.number_input('EstimatedReturn:', min_value=-0.2, max_value=0.4, step=1.,format="%.2f")
        LoanMonthsSinceOrigination = st.number_input('LoanMonthsSinceOrigination:', min_value=0, max_value=100)
        MonthlyLoanPayment = st.number_input('MonthlyLoanPayment:', min_value=0.0, max_value=8.0, step=1.,format="%.2f")
        
        LP_CustomerPayments = st.number_input('LP_CustomerPayments:', min_value=-2.0, max_value=41000.0, step=1.,format="%.2f")
        LP_CollectionFees = st.number_input('LP_CollectionFees:', min_value=-95.0, max_value=25000.0, step=1.,format="%.2f")
        LP_GrossPrincipalLoss = st.number_input('LP_GrossPrincipalLoss:', min_value=-954.0, max_value=25000.0, step=1.,format="%.2f")
        LP_NetPrincipalLoss = st.number_input('LP_NetPrincipalLoss:', min_value=-2.0, max_value=41000.0, step=1.,format="%.2f")
        DateCreditPulled_Year = st.selectbox('DateCreditPulled_Year:',[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015])    
    
        button = st.form_submit_button('Predict')
       
    
    with open('scaler.pkl', 'rb') as f:  # open a model file
            scaler = pickle.load(f) # deserialize the list

    if button:

        data= np.array([DateCreditPulled, LoanOriginationQuarter, BorrowerAPR, BorrowerRate, EstimatedEffectiveYield,
                        EstimatedReturn, LoanMonthsSinceOrigination, MonthlyLoanPayment,LP_CustomerPayments,
                        LP_CollectionFees, LP_GrossPrincipalLoss, LP_NetPrincipalLoss, DateCreditPulled_Year]).reshape(1, -1)
        
        features = scaler.transform(data)
        
        with open(models[model_names[option]], 'rb') as f:  # open a model file
            model = pickle.load(f) # deserialize the list
         
        pred = get_prediction(data=features, model=model)
        st.success(prediction(model.predict(data)))
        st.write(f"The predicted LoanStatus is:  {pred}") 
        
#         model = models[model_names[option]]
          
            


if __name__ == '__main__':
    main()        
        

Overwriting app.py


In [ ]:

import pickle
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from prediction import get_prediction, models




st.set_page_config(page_title='Prosper LoanStatus Prediction App', page_icon="💸",
                               layout="wide", initial_sidebar_state='expanded')

pickle_in = open('NB_model.pkl', 'rb') 
NB_model = pickle.load(pickle_in)

pickle_in = open('LG_model.pkl', 'rb') 
LG_model = pickle.load(pickle_in)

pickle_in = open('DT_model.pkl', 'rb') 
DT_model = pickle.load(pickle_in)



# creating option list for dropdown menu

features = ['ListingNumber', 'ClosedDate', 'BorrowerAPR', 'BorrowerRate','DateCreditPulled', 'LoanMonthsSinceOrigination', 'LoanNumber',
       'LoanOriginationQuarter', 'LP_CustomerPayments','LP_CustomerPrincipalPayments']

st.markdown("<h1 style='color:white ;text-align: center;'>Prosper LoanStatus Prediction App 💸 </h1>", unsafe_allow_html=True)


def prediction(num):
    if num ==0:
        return "Low Intensity"
    elif num==1:
        return "High Intensity"


def main():
    with st.form('prediction_form'):

        st.header("Predict the input for following features:") 
                
        activities=['Logistic Regression','Naives Bayes', 'KNN', 'Decision Tree', 'Random Forest', 'Ada Boost']
        model_names = {name:model_name for name, model_name in zip(activities, models.keys())}
        option=st.sidebar.selectbox('Which model would you like to use?' ,activities)
        st.write('You selected:', activities)
        st.subheader(option)
        
        ListingNumber = st.selectbox( 'ListingNumber:', [1,4,6])
        
        ClosedDate = st.selectbox( 'ClosedDate:', [1,78,81,92,100,105])
        BorrowerAPR = st.selectbox('BorrowerAPR:', [1,747,902,1260,1644,3672])
        BorrowerRate = st.selectbox( 'BorrowerRate:', [1,1319,1508,1651,1905,3672])
        DateCreditPulled = st.selectbox('DateCreditPulled:', [1,4,6])
        LoanMonthsSinceOrigination = st.selectbox('LoanMonthsSinceOrigination:', [8,9,13,20,4336,4485,4899,5215,5865])
        LoanNumber = st.selectbox('LoanNumber:', [1,4,6])
        LoanOriginationQuarter = st.selectbox('LoanOriginationQuarter:', [13,585,1243,1270,1600,2403,3074,3913,4424,5061,14450])
        LP_CustomerPayments = st.selectbox('LP_CustomerPayments:', [1,82,83,119,6208])
        LP_CustomerPrincipalPayments = st.selectbox('LP_CustomerPrincipalPayments:', [1,1938,2042,2274,2595,6308])
        button = st.form_submit_button('Predict')
       
    if button:

        data= np.array([ListingNumber, ClosedDate, BorrowerAPR, BorrowerRate, DateCreditPulled,LoanMonthsSinceOrigination,LoanNumber,
                        LoanOriginationQuarter,LP_CustomerPayments,LP_CustomerPrincipalPayments]).reshape(1, -1)
        
#         pred = get_prediction(data=data, model=LG_model)
#         pred = get_prediction(data=data, model=DT_model)
#         pred = get_prediction(data=data, model=NB_model)
        

#         if option == 'Logistic Regression':
#             st.success(prediction(LG_model.predict(data)))
#         elif option == 'DecisionTree':
#             st.success(prediction(DT_model.predict(data)))
#         else:
#             st.success(prediction(NB_model.predict(data)))    
        model = models[model_names[option]]
        pred = get_prediction(data=data, model=model)
        st.success(prediction(model.predict(data)))
        st.write(f"The predicted LoanStatus is:  {pred}")   
            
        


if __name__ == '__main__':
    main()        
        